## 18.3 Homework

Try different minimization methods in scipy on larger systems ($N$ up to 20), and show 
- 1 the average number of attempts to find the ground state
- 2 the time costs



### Optional
try to improve the code to make it run faster, analyze the most time consuming part and give your solution
    write function that get analytic gradient of LJ and put it into the hessien and run the code

In [56]:
import numpy as np
from numba import jit

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)
    
    # get rid of these for loops by using einsum!
    
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += LJ(dist)
    return E

@jit
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))



In [63]:
from scipy.optimize import minimize
import time

N = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
trueE = [-9.103852,-12.712062,-16.505384,-19.821489,
         -24.113360,-28.422532,-32.765970,-37.967600,
         -44.326801,-47.845157,-52.322627,-56.815742,
         -61.317995,-66.530949,-72.659782,-77.177043]

_method = ['BFGS',
           'CG',
           'L-BFGS-B']

threshold = 200
for N_atom in N:
    # tot_steps = 0
    for m in _method:
        print('%s (%s)' %(m, N_atom))
        f_values = 0
        count = 0
        init_time = time.time()
        while(f_values - trueE[N_atom-5] > 1e-2 and count < threshold):
            pos = init_pos(N_atom)
            res = minimize(total_energy, pos, method=m, tol=1e-4)
            f_values = res.fun
            x_values.append(res.x)
            count += 1 
            print('\r Step: {:d} value: {:.4f} Time: {:.2f} s'
                  .format(count,res.fun,time.time()-init_time),flush=True,end='') 
        # tot_steps += count
        print('\n loss', f_values - trueE[N_atom-5])


BFGS (5)
 Step: 1 value: -9.1039 Time: 0.03 s
 loss -4.156041004677036e-07
CG (5)
 Step: 1 value: -9.1039 Time: 0.04 s
 loss -4.156355846163251e-07
L-BFGS-B (5)
 Step: 13 value: -9.1038 Time: 0.13 s
 loss 9.584895023095896e-05
BFGS (6)
 Step: 44 value: -12.7121 Time: 1.88 s
 loss -2.5679096182784633e-07
CG (6)
 Step: 26 value: -12.3029 Time: 1.66 s

KeyboardInterrupt: 